In [43]:
import os
import requests
from datetime import datetime
import pandas as pd

In [11]:
# Weather API Details
weather_api_key = os.environ['DARKSKY_KEY']
weather_api_url = "https://api.darksky.net/forecast/"

# ZipCode to Long, Lat API Details
location_api_key = os.environ['ZIPCODEAPI_KEY']
location_api_url = "https://www.zipcodeapi.com/rest/"   

In [12]:
def get_location_for_zip(zipcode):
    url = location_api_url + location_api_key + '/info.json/' + str(zipcode).strip() + '/degrees'
    print('fetching {}'.format(url))
    return requests.get(url).json()

In [13]:
def get_forecast(lon, lat):
    url = weather_api_url + weather_api_key + '/{},{}'.format(lat, lon)
    print('fetching {}'.format(url))
    return requests.get(url).json()

In [15]:
location_zipcodes = ["23250"]  # 23250 Richmond Airport

In [34]:
forecasts = get_forecast_for_zips(location_zipcodes)

In [40]:
def forecast_rows(zipcodes):
    date_fetched = datetime.now()
    for zipcode in zipcodes:
        ll = get_location_for_zip(zipcode)
        forecast = get_forecast(ll['lng'], ll['lat'])
        for day in forecast['daily']['data']:
            yield {
                'source': 'Darksky',
                'date_fetched': date_fetched,
                'date': datetime.fromtimestamp(day['time']),
                'zipcode': zipcode,
                'latitude': ll['lat'],
                'longitude': ll['lng'],
                'tmax': day['temperatureHigh'],
                'tmin': day['temperatureLow'],
                'humidity': day['humidity'],
                'wind_speed': day['windSpeed'],
                'pressure': day['pressure'],
                'precip_total': day['precipIntensityMax'],
                'precip_prob': day['precipProbability'],
                'sunrise': datetime.fromtimestamp(day['sunriseTime']),
                'sunset': datetime.fromtimestamp(day['sunsetTime']),
                'cloud_cover': day['cloudCover'],
                'heat_index': day['apparentTemperatureHigh']
            }

In [46]:
forecast_data = pd.DataFrame(forecast_rows(location_zipcodes))

fetching https://www.zipcodeapi.com/rest/XJyHXErW0Xzm6WqyPPJDZrlSKse2jlHh4gY2UqYUoyhvuI3kxNXwejpZMD25ZJm3/info.json/23250/degrees
fetching https://api.darksky.net/forecast/0ab192d88a91b5cbf1904ee551f58b2c/37.504787,-77.320651


In [47]:
forecast_data.iloc[:3]

,cloud_cover,date,date_fetched,heat_index,humidity,latitude,longitude,precip_prob,precip_total,pressure,source,sunrise,sunset,tmax,tmin,wind_speed,zipcode
0,0.48,2017-12-22,2017-12-22 15:56:54.269976,59.75,0.73,37.504787,-77.320651,0.06,0.0098,1022.55,Darksky,2017-12-22 07:22:10,2017-12-22 16:56:22,59.75,46.32,2.92,23250
1,0.72,2017-12-23,2017-12-22 15:56:54.269976,69.75,0.78,37.504787,-77.320651,0.44,0.0302,1013.96,Darksky,2017-12-23 07:22:37,2017-12-23 16:56:53,69.75,42.20,9.47,23250
2,0.74,2017-12-24,2017-12-22 15:56:54.269976,43.54,0.70,37.504787,-77.320651,0.15,0.0012,1018.92,Darksky,2017-12-24 07:23:01,2017-12-24 16:57:26,46.23,37.82,5.38,23250
